In [39]:
import pandas as pd
from sqlalchemy import create_engine
from config import password

In [40]:
#Dependencies to scrap data
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import numpy as np

### Extract CSVs into DataFrames

In [54]:
car_file_path = "../top_selling_car.csv"
car_data = pd.read_csv(car_file_path)
car_data.head()

,#,Car brand,Car model,Release year,"Models sold, in thousands","Maximum speed, km/h",Percent of sells among the brand,"Sales in 2018, in thousands","Price, $"
0,1,Toyota,Corolla,1966,43000,220,15,1200,12000
1,2,Ford,F-Series,1977,40000,210,8,400,27000
2,3,Volkswagen,Golf,1974,30000,240,20,2500,18000
3,4,Volkswagen,Beetle,1938,23000,254,15,259,15000
4,5,Vaz,2101,1970,19000,180,30,10,12000


In [57]:
car_info = car_data[["#","Car brand","Car model","Release year","Maximum speed, km/h"]]
car_info = car_info.rename(columns ={"#":"ranking","Car brand":"brand","Car model":"model","Release year":"year_release","Maximum speed, km/h":"max_speed"})
car_info.head()

,ranking,brand,model,year_release,max_speed
0,1,Toyota,Corolla,1966,220
1,2,Ford,F-Series,1977,210
2,3,Volkswagen,Golf,1974,240
3,4,Volkswagen,Beetle,1938,254
4,5,Vaz,2101,1970,180


In [74]:
car_brand = car_data[["Car brand","Car model"]]
car_brand.loc[:,"year_2012"]="2012"
car_brand.head()

,Car brand,Car model,year_2012
0,Toyota,Corolla,2012
1,Ford,F-Series,2012
2,Volkswagen,Golf,2012
3,Volkswagen,Beetle,2012
4,Vaz,2101,2012


### Use data from the car_brand table to look for a number of cars on first page of each  model from Edmunds.com.

In [75]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#Url to get the good deal of cars
brand=car_brand["Car brand"].astype(str)
model=car_brand["Car model"].astype(str)
year=car_brand["year_2012"].astype(str)


In [51]:
new_data  = list() 
for i in range(0,len(brand)):
    try:
        url="https://www.edmunds.com/inventory/srp.html?inventorytype=used%2Ccpo&make="+brand[i]+"&model="+model[i]+"&year="+year[i]+"-"+year[i]
        browser.visit(url)
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')
        prices = soup.find_all('h3', class_='mb-0 text-gray-darker')
        results = soup.find_all('li', class_="search-results-inventory-card no-gutters list-unstyled bg-white mb-0_75 mb-lg-1 srp-vehicle-card hover row")
        for j in range(0,len(results)):
            data={}
            href=results[j].a["href"]
            desc=results[j].figure.img["alt"]
            vin=href.split('vin/')[1].split('/')[0]
            price = prices[j].text            
            mileage=results[j].find('span', class_="text-gray-darker ml-0_25").text
            if (desc and vin and mileage and price):
                data["model"]=model[i]
                data["description"]=desc
                data["vin"]=vin
                data["mileage"]=mileage
                data["price"]=price
                new_data.append(data)
    except (Exception, KeyError) as e:
        print(e)
# Close the browser after scraping
browser.quit() 

### Create a new DataFrame from the web scraping data

In [86]:
carweb_df=pd.DataFrame(new_data,columns=["model","description","vin","mileage","price"])
carweb_df.head()

,model,description,vin,mileage,price
0,Corolla,2012 Toyota Corolla LE in Silver,5YFBU4EE4CP018715,"113,119 miles","$6,995"
1,Corolla,2012 Toyota Corolla L in Dark Blue,5YFBU4EE4CP063136,"84,169 miles","$9,019"
2,Corolla,2012 Toyota Corolla S in Black,5YFBU4EE0CP029405,"138,476 miles","$7,995"
3,Corolla,2012 Toyota Corolla S in White,2T1BU4EE0CC900545,"56,752 miles","$12,900"
4,Golf,2012 Volkswagen Golf 2.0L TDI in Black,WVWDM7AJ0CW132117,"58,723 miles","$8,991"


### Connect to local database

In [78]:
connection_string = "postgres:"+ password+ "@localhost:5432/cars_db"
engine = create_engine(f'postgresql://{connection_string}')

### Check for tables

In [87]:
engine.table_names()

['car_info', 'car_web']

### Use pandas to load car_info DataFrame into database

In [82]:
car_info.to_sql(name='car_info', con=engine, if_exists='append', index=False)

### Use pandas to load carweb_df DataFrame into database

In [84]:
carweb_df.to_sql(name='car_web', con=engine, if_exists='append', index=False)

### Query data from car_brand table in Postgres SQL into Pandas

In [85]:
pd.read_sql_query('select * from car_info', con=engine).head()

,ranking,brand,model,year_release,max_speed
0,1,Toyota,Corolla,1966,220
1,2,Ford,F-Series,1977,210
2,3,Volkswagen,Golf,1974,240
3,4,Volkswagen,Beetle,1938,254
4,5,Vaz,2101,1970,180


### Query data from car_web table in Postgres SQL into Pandas

In [88]:
pd.read_sql_query('select * from car_web', con=engine).head()

,model,description,vin,mileage,price
0,Corolla,2012 Toyota Corolla LE in Silver,5YFBU4EE4CP018715,"113,119 miles","$6,995"
1,Corolla,2012 Toyota Corolla L in Dark Blue,5YFBU4EE4CP063136,"84,169 miles","$9,019"
2,Corolla,2012 Toyota Corolla S in Black,5YFBU4EE0CP029405,"138,476 miles","$7,995"
3,Corolla,2012 Toyota Corolla S in White,2T1BU4EE0CC900545,"56,752 miles","$12,900"
4,Golf,2012 Volkswagen Golf 2.0L TDI in Black,WVWDM7AJ0CW132117,"58,723 miles","$8,991"


### Query the data from two tables in PostgresSQL into pandas

In [122]:
query_table = pd.read_sql_query('select i.ranking, i.brand, i.max_speed, w.* from car_info i, car_web w where i.model=w.model', con=engine)
query_table

,ranking,brand,max_speed,model,description,vin,mileage,price
0,1,Toyota,220,Corolla,2012 Toyota Corolla S in White,2T1BU4EE0CC900545,"56,752 miles","$12,900"
1,1,Toyota,220,Corolla,2012 Toyota Corolla S in Black,5YFBU4EE0CP029405,"138,476 miles","$7,995"
2,1,Toyota,220,Corolla,2012 Toyota Corolla L in Dark Blue,5YFBU4EE4CP063136,"84,169 miles","$9,019"
3,1,Toyota,220,Corolla,2012 Toyota Corolla LE in Silver,5YFBU4EE4CP018715,"113,119 miles","$6,995"
4,3,Volkswagen,240,Golf,2012 Volkswagen Golf 2.0L TDI in Gray,WVWNM7AJ6CW162033,"77,155 miles","$13,000"
5,3,Volkswagen,240,Golf,2012 Volkswagen Golf 2.0L TDI in White,WVWMM7AJ9CW229008,"34,129 miles","$15,400"
6,3,Volkswagen,240,Golf,2012 Volkswagen Golf 2.0L TDI in Light Blue,WVWDM7AJ5CW337397,"63,183 miles","$13,900"
7,3,Volkswagen,240,Golf,2012 Volkswagen Golf 2.0L TDI,WVWBM7AJ0CW078339,"25,466 miles","$15,500"
8,3,Volkswagen,240,Golf,2012 Volkswagen Golf 2.0L TDI in Silver,WVWDM7AJ4CW345619,"69,833 miles","$11,091"
9,3,Volkswagen,240,Golf,2012 Volkswagen Golf 2.0L TDI in Silver,WVWNM7AJ2CW248052,"68,137 miles","$13,191"


In [121]:
pd.read_sql_query("select * from car_web w where model='Corolla'", con=engine)

,model,description,vin,mileage,price
0,Corolla,2012 Toyota Corolla LE in Silver,5YFBU4EE4CP018715,"113,119 miles","$6,995"
1,Corolla,2012 Toyota Corolla L in Dark Blue,5YFBU4EE4CP063136,"84,169 miles","$9,019"
2,Corolla,2012 Toyota Corolla S in Black,5YFBU4EE0CP029405,"138,476 miles","$7,995"
3,Corolla,2012 Toyota Corolla S in White,2T1BU4EE0CC900545,"56,752 miles","$12,900"


In [127]:
pd.read_sql_query("select * from car_web where model ='Civic'", con=engine)

,model,description,vin,mileage,price
0,Civic,2012 Honda Civic LX in Light Blue,2HGFG3B58CH561239,"6,055 miles","$9,995"
1,Civic,2012 Honda Civic Si in Black,2HGFG4A51CH701073,"90,066 miles","$9,991"
2,Civic,2012 Honda Civic EX in Silver,2HGFG3B89CH542385,"88,372 miles","$6,495"
3,Civic,2012 Honda Civic Si in White,2HGFG4A51CH705625,"85,802 miles","$11,795"
4,Civic,2012 Honda Civic Si in Red,2HGFG4A59CH705355,"73,558 miles","$12,750"
5,Civic,2012 Honda Civic LX in Silver,2HGFB2F54CH543503,"123,392 miles","$8,413"
6,Civic,2012 Honda Civic EX,2HGFG3B83CH529387,"32,466 miles","$13,998"
7,Civic,2012 Honda Civic LX in Gray,2HGFB2F54CH315470,"112,837 miles","$7,675"
8,Civic,2012 Honda Civic LX in Silver,19XFB2F52CE007241,"110,650 miles","$8,265"
9,Civic,2012 Honda Civic Si in Black,2HGFB6E54CH700567,"118,555 miles","$10,495"
